# Test for evaluating the invariant law for different $\sigma$ values in relative $I$ evolution

## Setup scripts if we are under SWAN

In [ ]:
# Working in the right path
%cd /eos/project/d/da-and-diffusion-studies/DA_Studies/Simulations/Models/loss_studies/notebooks

In [ ]:
# Install the libraries
import sys
!{sys.executable} -m pip install --user tqdm pynverse sixtrackwrap crank-nicolson-numba henon-map symplectic-map
!{sys.executable} -m pip install --user --upgrade sixtrackwrap 
!{sys.executable} -m pip install --user --upgrade crank-nicolson-numba 
!{sys.executable} -m pip install --user --upgrade henon-map 
!{sys.executable} -m pip install --user --upgrade symplectic-map
!export PYTHONPATH=$CERNBOX_HOME.local/lib/python3.7/site-packages:$PYTHONPATH

In [ ]:
# For this "presentation" only!
import warnings
warnings.filterwarnings('ignore')

## Imports

In [2]:
%matplotlib widget

In [3]:
# Base libraries
import math
import numpy as np
import scipy.integrate as integrate
from scipy.special import erf
import pickle
import itertools
from scipy.optimize import curve_fit

from numba import njit, prange

from tqdm.notebook import tqdm
import time
import matplotlib.pyplot as plt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.ticker as ticker
from math import gcd

import pandas as pd

from scipy.special import lambertw
from scipy.interpolate import interp1d

import os

# Personal libraries
import sixtrackwrap as sx
import crank_nicolson_numba.nekhoroshev as nk

# Personal modules
import fit_utils as fit

/home/carlidel/Insync/carlo.montanari3@studio.unibo.it/OneDrive Biz/projects/loss_studies/notebooks/fit_utils.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


## Load data and setup original DA

In [6]:
sigma = 0.3
sigma_b = 0.5

In [5]:
savepath = "../data/"

turn_samples = 100
min_turns = 10**3
max_turns = 10**6

# Do we want to load ALL the Hénon data files or should we skip some?
skipper = 10

cut_point = 1.0

In [9]:
epsilons = []

real_losses = []
gaussian_losses = []

real_DAs = []
gaussian_DAs = []

real_losses_b = []
gaussian_losses_b = []
gaussian_DAs_b = []

files = list(sorted(list(filter(lambda f: "henon_eps_" in f and "unif" not in f, os.listdir(savepath))), key=lambda f: float(f[10: -4])))[::skipper]

for f in tqdm(files):
    print("Loading file: ", f)
    unif_f = "unif_" + f
    epsilon = float(f[10: -4])
    epsilons.append(epsilon)
    engine = sx.uniform_radial_scanner.load_values(savepath + f)
    unif_engine = sx.uniform_scanner.load_values(savepath + unif_f)
    
    baseline_samples = engine.baseline_samples
    d_r = engine.dr
    turn_sampling = np.linspace(min_turns, max_turns, turn_samples, dtype=np.int)[::-1]
    
    print("Real part")
    real_DAs.append(engine.compute_DA_standard(turn_sampling))
    print("Done DA")
    
    print("Gaussian part")
    engine.assign_weights(sx.assign_symmetric_gaussian(sigma))
    unif_engine.assign_weights(
        sx.assign_symmetric_gaussian(sigma, polar=False), radial_cut=cut_point)
    baseline = engine.compute_loss_cut(cut_point)
    
    gaussian_losses.append(engine.compute_loss(turn_sampling, cut_point, True))
    gaussian_DAs.append(
        fit.DA_from_symmetric_gaussian_loss(gaussian_losses[-1], sigma, cut_point))
    print("Done Gaussian")
    
    print("Gaussian loss but with 'Real' DA values")
    real_losses.append(fit.symmetric_gaussian_loss(real_DAs[-1], sigma, cut_point))
    
    ##############################################################################
    print("Bis")
    print("Gaussian part")
    engine.assign_weights(sx.assign_symmetric_gaussian(sigma_b))
    unif_engine.assign_weights(
        sx.assign_symmetric_gaussian(sigma_b, polar=False), radial_cut=cut_point)
    baseline = engine.compute_loss_cut(cut_point)
    
    gaussian_losses_b.append(engine.compute_loss(turn_sampling, cut_point, True))
    gaussian_DAs_b.append(
        fit.DA_from_symmetric_gaussian_loss(gaussian_losses_b[-1], sigma_b, cut_point))
    print("Done Gaussian")
    
    print("Gaussian loss but with 'Real' DA values")
    real_losses_b.append(fit.symmetric_gaussian_loss(real_DAs[-1], sigma_b, cut_point))
    
    print("Finished processing.")

Loading file:  henon_eps_2.pkl
Real part
Done DA
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Bis
Gaussian part
Done Gaussian
Gaussian loss but with 'Real' DA values
Finished processing.



In [25]:
sigma = 0.2
sigma_b = 0.3

engine.assign_weights(sx.assign_symmetric_gaussian(sigma))
loss = engine.compute_loss([turn_sampling[50]], cut_point, True)
print(fit.DA_from_symmetric_gaussian_loss(loss, sigma, cut_point))

engine.assign_weights(sx.assign_symmetric_gaussian(sigma_b))
loss = engine.compute_loss([turn_sampling[50]], cut_point, True)
print(fit.DA_from_symmetric_gaussian_loss(loss, sigma_b, cut_point))

unif_engine.assign_weights(sx.assign_symmetric_gaussian(sigma, polar=False), radial_cut=cut_point)
loss = unif_engine.compute_loss([turn_sampling[50]], True)
print(fit.DA_from_symmetric_gaussian_loss(loss, sigma, cut_point))

unif_engine.assign_weights(sx.assign_symmetric_gaussian(sigma_b, polar=False), radial_cut=cut_point)
loss = unif_engine.compute_loss([turn_sampling[50]], True)
print(fit.DA_from_symmetric_gaussian_loss(loss, sigma_b, cut_point))

[0.48725754]
[0.49529021]
[0.48380637]
[0.49071474]


In [16]:
plt.figure()

for i in range(len(real_losses)):
    plt.plot(turn_sampling, real_losses[i], c="C0")
    plt.plot(turn_sampling, gaussian_losses[i], c="C1")
    plt.plot(turn_sampling, real_losses_b[i], c="C0")
    plt.plot(turn_sampling, gaussian_losses_b[i], c="C1")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.figure()

for i in range(len(real_losses)):
    plt.plot(turn_sampling, real_DAs[i], c="C0")
    plt.plot(turn_sampling, gaussian_DAs[i], c="C1")
    plt.plot(turn_sampling, gaussian_DAs_b[i], c="C2")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …